In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import scipy.io
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
root_dir = '../input/iris-encode/template_train'
X_train = []
y_train = []
labels = os.listdir(root_dir)
labels.sort()
for label in labels:
    mat = scipy.io.loadmat(os.path.join(root_dir,label))
    X_train.append(mat['template'].flatten().astype('bool'))
    y_train.append(mat['label'][0])

root_dir = '../input/iris-encode/template_test'
X_test = []
y_test = []
labels = os.listdir(root_dir)
labels.sort()
for label in labels:
    mat = scipy.io.loadmat(os.path.join(root_dir,label))
    X_test.append(mat['template'].flatten().astype('bool'))
    y_test.append(mat['label'][0])
                  
                  
X_train = pd.DataFrame(X_train)                   
X_test = pd.DataFrame(X_test)    
y_train = LabelEncoder().fit_transform(y_train)
y_test =  LabelEncoder().fit_transform(y_test)

In [ ]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgbm
import catboost as cb

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
SEED = 123
cross_valid_scores = {}
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, make_scorer

Logistic regression

In [ ]:
%%time
parameters = {
    "C": [ 0.1, 1.,10, 50],
    "penalty": [ "l1","l2"]
}
SEED = 123
model_logistic_regression = LogisticRegression(
    random_state=SEED,
    class_weight="balanced",
    solver="liblinear",
    
)

model_logistic_regression = GridSearchCV(
    model_logistic_regression, 
    parameters, 
    cv=StratifiedKFold(n_splits=5),
    scoring= 'roc_auc_ovr',
    verbose = 10,
)

model_logistic_regression.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_logistic_regression.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_logistic_regression.best_score_:.3f}'
)
cross_valid_scores['logistic_regression'] = model_logistic_regression.best_score_
print('-----')


In [ ]:
res = pd.concat([pd.DataFrame(model_logistic_regression.cv_results_["params"]),pd.DataFrame(model_logistic_regression.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_logistic_regression.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_logistic_regression.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res


Decision Tree Classifier


In [ ]:
%%time



parameters = {
    "max_depth": [3, 5, 21,51,101,181,300,450, 600],
}

model_desicion_tree = DecisionTreeClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_desicion_tree = GridSearchCV(
    model_desicion_tree, 
    parameters, 
    cv=5,
    scoring='f1_macro',
    verbose = 10,
)

model_desicion_tree.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_desicion_tree.best_params_}')
print(
    f'Mean cross-validated  score of the best_estimator: ' + \
    f'{model_desicion_tree.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_desicion_tree.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_desicion_tree.cv_results_["params"]),pd.DataFrame(model_desicion_tree.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_desicion_tree.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_desicion_tree.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

Random Forest Classifier

In [ ]:

%%time
parameters = {
    "n_estimators": [1800], 
    "max_depth": [   11],
}

model_random_forest = RandomForestClassifier(
    random_state=SEED,
    class_weight='balanced',
)

model_random_forest = GridSearchCV(
    model_random_forest, 
    parameters, 
    cv=StratifiedKFold(n_splits=5),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_random_forest.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_random_forest.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_random_forest.best_score_:.3f}'
)
cross_valid_scores['random_forest'] = model_random_forest.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_random_forest.cv_results_["params"]),pd.DataFrame(model_random_forest.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_random_forest.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_random_forest.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

ADABOOST

In [ ]:
%%time
parameters = {
    "n_estimators": [  300, 400, 500], 
    "learning_rate": [  1.1, 1.2, 1.3 ],
}

model_adaboost = AdaBoostClassifier(
    random_state=SEED,
)

model_adaboost = GridSearchCV(
    model_adaboost, 
    parameters, 
    cv=StratifiedKFold(n_splits=5,shuffle = True, random_state= SEED),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_adaboost.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_adaboost.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{model_adaboost.best_score_:.3f}'
)
cross_valid_scores['ada_boost'] = model_adaboost.best_score_
print('-----')

In [ ]:

res = pd.concat([pd.DataFrame(model_adaboost.cv_results_["params"]),pd.DataFrame(model_adaboost.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_adaboost.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_adaboost.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

SVC

In [ ]:
%%time
parameters = {
    "C": [ 20, 50,100],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
}

model_svc = SVC(
    random_state=SEED,
    class_weight="balanced",
    probability=True,
)

model_svc = GridSearchCV(
    model_svc, 
    parameters, 
    cv=StratifiedKFold(n_splits=5),
    scoring='roc_auc_ovr',
    verbose = 10,
)

model_svc.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_svc.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_svc.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_svc.best_score_
print('-----')

In [ ]:
res = pd.concat([pd.DataFrame(model_svc.cv_results_["params"]),pd.DataFrame(model_svc.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_svc.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_svc.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

GaussianNB

In [ ]:
%%time
parameters =  {'var_smoothing': np.logspace(0,-9, num=100)}

model_gaussianNB = GaussianNB()

model_gaussianNB = GridSearchCV(
    model_gaussianNB, 
    parameters, 
    cv=StratifiedKFold(n_splits=5),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
)

model_gaussianNB.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_gaussianNB.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_gaussianNB.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_gaussianNB.best_score_
print('-----')

In [ ]:
res = pd.concat([pd.DataFrame(model_gaussianNB.cv_results_["params"]),pd.DataFrame(model_gaussianNB.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_gaussianNB.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_gaussianNB.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res

KNeighborsClassifier


In [ ]:
%%time
parameters =    {
        'n_neighbors': [3, 5, 7, 9] # usually odd numbers
    }

model_KNN =  KNeighborsClassifier()

model_KNN = GridSearchCV(
    model_KNN, 
    parameters, 
    cv=StratifiedKFold(n_splits=5),
    scoring='roc_auc_ovr',
    verbose = 10,
    n_jobs = -1
 
)

model_KNN.fit(X_train, y_train)

print('-----')
print(f'Best parameters {model_KNN.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_KNN.best_score_:.3f}'
)
cross_valid_scores['svc'] = model_KNN.best_score_
print('-----')

res = pd.concat([pd.DataFrame(model_KNN.cv_results_["params"]),pd.DataFrame(model_KNN.cv_results_["mean_test_score"], columns=["AUC_ROC"])],axis=1)

y_pred = model_KNN.best_estimator_.predict( X_test)
print(accuracy_score(y_test, y_pred))
y_pred_proba = model_KNN.best_estimator_.predict_proba( X_test)
print(roc_auc_score(y_test, y_pred_proba, multi_class = 'ovr'))
res